In [1]:
import numpy as np
import random
import json
import os
import time
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Embedding
import tensorflow.contrib.legacy_seq2seq as seq2seq
from sklearn.model_selection import train_test_split
print ("Packages Imported")

Using TensorFlow backend.


Packages Imported


## Load Data: English2French

In [2]:
data_dir = "en_fr"
input_file_en = os.path.join(data_dir, "A_53_647_CORR1_en.snt")
lines_en = []
with open(input_file_en, "r", encoding="utf-8") as f_en:
    for line in f_en.readlines():
        line = line.replace('\n','')
        if (not line == ''):
            lines_en.append(line)
        
print(lines_en[-2])

Figures now represent the initial appropriation for each biennium, net of staff assessment, divided into two equal annual amounts.


In [3]:
data_dir = "en_fr"
input_file_fr = os.path.join(data_dir, "A_53_647_CORR1_fr.snt")
lines_fr = []
with open(input_file_fr, "r", encoding="utf-8") as f_fr:
    for line in f_fr.readlines():
        line = line.replace('\n','')
        if (not line == ''):
            lines_fr.append(line)
        
print(lines_fr[-2])

0044165f.doc 2


In [4]:
lines_en

['Fifty-third session',
 'Agenda item 116',
 'Administrative and budgetary coordination of the United Nations with the specialized agencies and the International Atomic Energy Agency',
 'Budgetary and financial situation of organizations of the United Nations system',
 'Statistical report by the Administrative Committee on\xa0Coordination',
 'Corrigendum',
 'Table 1.',
 'Approved regular budgets (1990-1999) (see p. 2) have been revised with respect to the figures originally reported for the United Nations in order to present comparable figures from one biennium to the next.',
 'Figures now represent the initial appropriation for each biennium, net of staff assessment, divided into two equal annual amounts.',
 'Pages 2 and 3 for offset.']

In [5]:
lines_fr[:10]

['Cinquante-troisième session',
 "Point 116 de l'ordre du jour",
 "Coordination administrative et budgétaire entre l'Organisation des Nations Unies et les institutions spécialisées ainsi que l'Agence internationale de l'énergie atomique",
 'Situation budgétaire et financière des organismes des Nations Unies',
 'Rapport statistique du Comité administratif de coordination',
 'Rectificatif',
 'Tableau 1.',
 "On a révisé les chiffres initialement indiqués pour l'ONU dans les budgets ordinaires approuvés, 1990-1999 (voir p.",
 "2), afin de présenter des chiffres comparables d'un exercice biennal à l'autre.",
 'Les nouveaux chiffres correspondent aux crédits initialement ouverts pour chaque exercice biennal, déduction faite des contributions du personnel, répartis en deux montants annuels égaux.']

## Preprocess Data

In [6]:
def preprocess_en():
    lines_en[7] = lines_en[7] + lines_en[8] + lines_en[9]
    lines_en[5] = 'Correction'
def preprocess_fr():
    lines_fr[7] = lines_fr[7] + lines_fr[8] + lines_fr[9]

In [7]:
preprocess_en()
preprocess_fr()

In [8]:
lines_en[:8]

['Fifty-third session',
 'Agenda item 116',
 'Administrative and budgetary coordination of the United Nations with the specialized agencies and the International Atomic Energy Agency',
 'Budgetary and financial situation of organizations of the United Nations system',
 'Statistical report by the Administrative Committee on\xa0Coordination',
 'Correction',
 'Table 1.',
 'Approved regular budgets (1990-1999) (see p. 2) have been revised with respect to the figures originally reported for the United Nations in order to present comparable figures from one biennium to the next.Figures now represent the initial appropriation for each biennium, net of staff assessment, divided into two equal annual amounts.Pages 2 and 3 for offset.']

In [9]:
lines_fr[:8]

['Cinquante-troisième session',
 "Point 116 de l'ordre du jour",
 "Coordination administrative et budgétaire entre l'Organisation des Nations Unies et les institutions spécialisées ainsi que l'Agence internationale de l'énergie atomique",
 'Situation budgétaire et financière des organismes des Nations Unies',
 'Rapport statistique du Comité administratif de coordination',
 'Rectificatif',
 'Tableau 1.',
 "On a révisé les chiffres initialement indiqués pour l'ONU dans les budgets ordinaires approuvés, 1990-1999 (voir p.2), afin de présenter des chiffres comparables d'un exercice biennal à l'autre.Les nouveaux chiffres correspondent aux crédits initialement ouverts pour chaque exercice biennal, déduction faite des contributions du personnel, répartis en deux montants annuels égaux."]

In [10]:
lines_en = lines_en[:8] * 20
lines_fr = lines_fr[:8] * 20

In [11]:
word_data = []
for i in range(len(lines_en)):
    word_data.append((lines_en[i],lines_fr[i]))

In [12]:
# word_data = [
#     ('how are you', 'Comment allez-vous'),
#     ('Want to go to Naver','Voulez-vous aller à Naver'),
#     ('I love drone', "J'adore drone")
# ] * 10

In [13]:
word_data[:8]

[('Fifty-third session', 'Cinquante-troisième session'),
 ('Agenda item 116', "Point 116 de l'ordre du jour"),
 ('Administrative and budgetary coordination of the United Nations with the specialized agencies and the International Atomic Energy Agency',
  "Coordination administrative et budgétaire entre l'Organisation des Nations Unies et les institutions spécialisées ainsi que l'Agence internationale de l'énergie atomique"),
 ('Budgetary and financial situation of organizations of the United Nations system',
  'Situation budgétaire et financière des organismes des Nations Unies'),
 ('Statistical report by the Administrative Committee on\xa0Coordination',
  'Rapport statistique du Comité administratif de coordination'),
 ('Correction', 'Rectificatif'),
 ('Table 1.', 'Tableau 1.'),
 ('Approved regular budgets (1990-1999) (see p. 2) have been revised with respect to the figures originally reported for the United Nations in order to present comparable figures from one biennium to the next.

In [14]:
x = [x for x, y in word_data]
y = [y for x, y in word_data]

In [15]:
x_list = []
for i in range(len(x)):
    x_text = x[i].split()
    x_list.append(x_text)
for i in range(1, len(x_list)):
    x_list[i] = set(x_list[i-1]).union(set(x_list[i]))
    x_words_set = x_list[len(x_list)-1]

In [16]:
y_list = []
for i in range(len(y)):
    y_text = y[i].split()
    y_list.append(y_text)
for i in range(1, len(y_list)):
    y_list[i] = set(y_list[i-1]).union(set(y_list[i]))
    y_words_set = y_list[len(y_list)-1]

In [17]:
word2numX = dict(zip(x_words_set, range(len(x_words_set))))
word2numY = dict(zip(y_words_set, range(len(y_words_set))))

In [18]:
print(word2numX)
print(word2numY)
print(x[0])
print(y[0])
print(y[4])

{'financial': 0, '2': 1, 'specialized': 2, 'by': 3, 'Committee': 4, 'staff': 5, 'figures': 6, 'Fifty-third': 7, 'report': 8, 'equal': 9, 'to': 10, 'net': 11, 'comparable': 12, 'reported': 13, 'Nations': 14, 'present': 15, 'Administrative': 16, 'Budgetary': 17, 'Statistical': 18, 'annual': 19, 'budgets': 20, 'from': 21, 'regular': 22, 'assessment,': 23, 'in': 24, 'Agenda': 25, 'the': 26, 'revised': 27, 'situation': 28, 'have': 29, 'item': 30, 'one': 31, 'p.': 32, 'and': 33, 'each': 34, '1.': 35, 'next.Figures': 36, 'International': 37, 'United': 38, '(1990-1999)': 39, '116': 40, 'offset.': 41, 'Table': 42, 'agencies': 43, 'order': 44, 'Energy': 45, 'Atomic': 46, 'organizations': 47, 'into': 48, 'been': 49, 'two': 50, 'coordination': 51, 'Coordination': 52, 'biennium': 53, 'with': 54, 'Approved': 55, 'Correction': 56, '(see': 57, 'system': 58, 'divided': 59, '2)': 60, 'initial': 61, 'biennium,': 62, 'now': 63, 'amounts.Pages': 64, 'of': 65, 'represent': 66, 'on': 67, 'budgetary': 68, 'fo

## Padding: All sequences that are shorter than the max length of the sequence

In [19]:
word2numX['<PAD_ENC>'] = len(word2numX)
num2wordX = dict(zip(word2numX.values(), word2numX.keys()))

In [20]:
# max_len 뽑음
numWordsList = []
for word in x:
    Words = word.split(' ')
    num = len(Words)
    numWordsList.append(num) 
max_len_enc = max(numWordsList)

In [21]:
step1_X = [
    word.split() for word in x
]

In [22]:
step1_X

[['Fifty-third', 'session'],
 ['Agenda', 'item', '116'],
 ['Administrative',
  'and',
  'budgetary',
  'coordination',
  'of',
  'the',
  'United',
  'Nations',
  'with',
  'the',
  'specialized',
  'agencies',
  'and',
  'the',
  'International',
  'Atomic',
  'Energy',
  'Agency'],
 ['Budgetary',
  'and',
  'financial',
  'situation',
  'of',
  'organizations',
  'of',
  'the',
  'United',
  'Nations',
  'system'],
 ['Statistical',
  'report',
  'by',
  'the',
  'Administrative',
  'Committee',
  'on',
  'Coordination'],
 ['Correction'],
 ['Table', '1.'],
 ['Approved',
  'regular',
  'budgets',
  '(1990-1999)',
  '(see',
  'p.',
  '2)',
  'have',
  'been',
  'revised',
  'with',
  'respect',
  'to',
  'the',
  'figures',
  'originally',
  'reported',
  'for',
  'the',
  'United',
  'Nations',
  'in',
  'order',
  'to',
  'present',
  'comparable',
  'figures',
  'from',
  'one',
  'biennium',
  'to',
  'the',
  'next.Figures',
  'now',
  'represent',
  'the',
  'initial',
  'appropri

In [23]:
second_list = []
for i in range(len(step1_X)):
    first_list = []
    for j in range(len(step1_X[i])):
        k = word2numX[step1_X[i][j]]
        first_list.append(k)
        padded_list = [word2numX['<PAD_ENC>']] * (max_len_enc - len(first_list)) + first_list
    second_list.append(padded_list)
x = second_list
x = np.array(x)

In [24]:
x[:10]

array([[76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76,  7, 70],
       [76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 25, 30, 40],
       [76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 16, 33, 68, 51, 65, 26, 38, 14, 54, 26,  2, 43, 33,
        26, 37, 46, 45, 74],
       [76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
        76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 17, 33,  0, 28, 65, 47,
 

In [25]:
word2numY['<GO>'] = len(word2numY)
word2numY['<PAD_DEC>'] = len(word2numY) 
num2wordY = dict(zip(word2numY.values(), word2numY.keys()))

In [26]:
# Extract max_len
numWordsList = []
for word in y:
    Words = word.split(' ')
    num = len(Words)
    numWordsList.append(num) 
max_len_dec = max(numWordsList)

In [27]:
max_len_dec

51

In [28]:
step1_Y = [
    word.split() for word in y
]

In [29]:
second_list = []
for i in range(len(step1_Y)):
    first_list_ = []
    for j in range(len(step1_Y[i])):
        k = word2numY[step1_Y[i][j]]
        first_list_.append(k)
        padded_list = [word2numY['<GO>']] + first_list_ + [word2numY['<PAD_DEC>']] * (max_len_dec + 1 - len(first_list_))
    second_list.append(padded_list)
y = second_list
y = np.array(y)

for z in range(len(second_list)-1,-1,-1):
    print(len(second_list[z]))

53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53


In [30]:
y.shape

(160, 53)

In [31]:
y

array([[77, 14, 71, ..., 78, 78, 78],
       [77, 43, 36, ..., 78, 78, 78],
       [77, 52, 29, ..., 78, 78, 78],
       ..., 
       [77,  0, 78, ..., 78, 78, 78],
       [77, 48, 32, ..., 78, 78, 78],
       [77, 25, 42, ..., 38, 27, 78]])

In [32]:
x_seq_length = len(x[0])
y_seq_length = len(y[0]) - 1
print(x_seq_length)
print(y_seq_length)

56
52


In [33]:
len(x)

160

In [34]:
len(y)

160

In [35]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [36]:

batch_size = 20
nodes = 256
embed_size = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()

inputs = tf.placeholder(tf.int32, (None, x_seq_length), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output') #
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers - For dynamic RNN
input_embedding = tf.Variable(tf.random_uniform((len(word2numX), embed_size), -1.0, 1.0), name='enc_embedding')
output_embedding = tf.Variable(tf.random_uniform((len(word2numY), embed_size), -1.0, 1.0), name='dec_embedding')
word_input_embed = tf.nn.embedding_lookup(input_embedding, inputs)
word_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

# We feed Encoder's last_state to decoder's initial state 
with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes) 
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=word_input_embed, dtype=tf.float32)

# dec_outputs are what we want to get    
with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=word_output_embed, initial_state=last_state)
        
#connect outputs
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(word2numY), activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function - softmax
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.AdamOptimizer().minimize(loss)

## Training

In [37]:
# X_train is input_sequence and y_train is output_sequence
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [38]:
# input_sequence is called source_batch
# targets is every words except for GO, and we feed targets as decoder input
# which is called teacher enforcing

sess.run(tf.global_variables_initializer())
epochs = 1000
for epoch_i in range(epochs):
    start_time = time.time()
    for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
        _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
            feed_dict = {inputs: source_batch,
                        outputs: target_batch[:, :-1],
                        targets: target_batch[:, 1:]})
    accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])

    print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                      accuracy, time.time() - start_time))

Epoch   0 Loss:  2.757 Accuracy: 0.7558 Epoch duration:  0.938s
Epoch   1 Loss:  1.114 Accuracy: 0.8077 Epoch duration:  0.918s
Epoch   2 Loss:  1.127 Accuracy: 0.7587 Epoch duration:  0.896s
Epoch   3 Loss:  0.810 Accuracy: 0.8327 Epoch duration:  0.885s
Epoch   4 Loss:  1.120 Accuracy: 0.7462 Epoch duration:  0.854s
Epoch   5 Loss:  1.328 Accuracy: 0.7038 Epoch duration:  0.867s
Epoch   6 Loss:  1.731 Accuracy: 0.6096 Epoch duration:  0.881s
Epoch   7 Loss:  0.832 Accuracy: 0.8019 Epoch duration:  0.849s
Epoch   8 Loss:  1.077 Accuracy: 0.7308 Epoch duration:  0.873s
Epoch   9 Loss:  0.657 Accuracy: 0.8260 Epoch duration:  0.856s
Epoch  10 Loss:  1.015 Accuracy: 0.7404 Epoch duration:  0.864s
Epoch  11 Loss:  0.461 Accuracy: 0.8683 Epoch duration:  0.855s
Epoch  12 Loss:  1.260 Accuracy: 0.6606 Epoch duration:  0.859s
Epoch  13 Loss:  0.922 Accuracy: 0.7346 Epoch duration:  0.873s
Epoch  14 Loss:  0.661 Accuracy: 0.8173 Epoch duration:  0.858s
Epoch  15 Loss:  0.772 Accuracy: 0.7750 

KeyboardInterrupt: 

In [39]:
source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))
dec_input = np.zeros((len(source_batch), 1)) + word2numY['<GO>']
print(dec_input.shape)
for i in range(y_seq_length):
    batch_logits = sess.run(logits,
                feed_dict = {inputs: source_batch, 
                 outputs: dec_input})  
    # We predict for maximum probability to be the maximum (One-hot)
    print(batch_logits.shape)
    prediction = batch_logits[:,-1].argmax(axis=-1)
    dec_input = np.hstack([dec_input, prediction[:,None]])
    
print('#####=====Test Accuracy: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

(20, 1)
(20, 1, 79)
(20, 2, 79)
(20, 3, 79)
(20, 4, 79)
(20, 5, 79)
(20, 6, 79)
(20, 7, 79)
(20, 8, 79)
(20, 9, 79)
(20, 10, 79)
(20, 11, 79)
(20, 12, 79)
(20, 13, 79)
(20, 14, 79)
(20, 15, 79)
(20, 16, 79)
(20, 17, 79)
(20, 18, 79)
(20, 19, 79)
(20, 20, 79)
(20, 21, 79)
(20, 22, 79)
(20, 23, 79)
(20, 24, 79)
(20, 25, 79)
(20, 26, 79)
(20, 27, 79)
(20, 28, 79)
(20, 29, 79)
(20, 30, 79)
(20, 31, 79)
(20, 32, 79)
(20, 33, 79)
(20, 34, 79)
(20, 35, 79)
(20, 36, 79)
(20, 37, 79)
(20, 38, 79)
(20, 39, 79)
(20, 40, 79)
(20, 41, 79)
(20, 42, 79)
(20, 43, 79)
(20, 44, 79)
(20, 45, 79)
(20, 46, 79)
(20, 47, 79)
(20, 48, 79)
(20, 49, 79)
(20, 50, 79)
(20, 51, 79)
(20, 52, 79)
#####=====Test Accuracy:  1.000


In [40]:
num_preds = 20
source_words = [[num2wordX[l] for l in sent if num2wordX[l]!="<PAD_ENC>"] for sent in source_batch[:num_preds]]
dest_words = [[num2wordY[l] for l in sent if num2wordY[l] != "<PAD_DEC>"]for sent in dec_input[:num_preds, 1:]]
# print(dec_input)
# print(dest_words)

for words_in, words_out in zip(source_words, dest_words):
    print('English======>',' '.join(words_in)+'\n'+'French ======>',' '.join(words_out),'\n')

English======> Administrative and budgetary coordination of the United Nations with the specialized agencies and the International Atomic Energy Agency
French ======> Coordination administrative et budgétaire entre l'Organisation des Nations Unies et les institutions spécialisées ainsi que l'Agence internationale de l'énergie atomique 

English======> Fifty-third session
French ======> Cinquante-troisième session 

English======> Statistical report by the Administrative Committee on Coordination
French ======> Rapport statistique du Comité administratif de coordination 

English======> Statistical report by the Administrative Committee on Coordination
French ======> Rapport statistique du Comité administratif de coordination 

English======> Table 1.
French ======> Tableau 1. 

English======> Budgetary and financial situation of organizations of the United Nations system
French ======> Situation budgétaire et financière des organismes des Nations Unies 

English======> Budgetary and fi